In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../../pre-miRNAs/filtered_premirna_from_S/vmir_output_from_S_filtered.csv')
df

,sequence,mfe,mfeis,class
0,ATGCGGAATTATATAGGACAGAATAATCAGCAACACAGTTGCTGAT...,-23.900000,-0.919231,Real
1,ATTGTGTGAATTTGGACATGTTCTTCAGGCTCATCAACAATTTTAT...,-25.000000,-0.892857,Real
2,GGTTTTATAGCTGGCTTGATTGCCATAGTAATGGTGACAATTATGC...,-21.500000,-0.895833,Real
3,TTGAACATGTTACCTTCTTCATCTACAATAAAATTGTTGATGAGCC...,-18.200001,-0.866667,Pseudo


In [3]:
complement_trans = str.maketrans('T', 'U')
for i in range(df.shape[0]):
    seq = df.iloc[i].sequence
    df.loc[i,'sequence'] = seq.translate(complement_trans)

df

,sequence,mfe,mfeis,class
0,AUGCGGAAUUAUAUAGGACAGAAUAAUCAGCAACACAGUUGCUGAU...,-23.900000,-0.919231,Real
1,AUUGUGUGAAUUUGGACAUGUUCUUCAGGCUCAUCAACAAUUUUAU...,-25.000000,-0.892857,Real
2,GGUUUUAUAGCUGGCUUGAUUGCCAUAGUAAUGGUGACAAUUAUGC...,-21.500000,-0.895833,Real
3,UUGAACAUGUUACCUUCUUCAUCUACAAUAAAAUUGUUGAUGAGCC...,-18.200001,-0.866667,Pseudo


In [4]:
filtered_premirnas = df[df['class'] == 'Real']
filtered_premirnas

,sequence,mfe,mfeis,class
0,AUGCGGAAUUAUAUAGGACAGAAUAAUCAGCAACACAGUUGCUGAU...,-23.9,-0.919231,Real
1,AUUGUGUGAAUUUGGACAUGUUCUUCAGGCUCAUCAACAAUUUUAU...,-25.0,-0.892857,Real
2,GGUUUUAUAGCUGGCUUGAUUGCCAUAGUAAUGGUGACAAUUAUGC...,-21.5,-0.895833,Real


In [12]:
!pip install bs4
!pip install tqdm

     |████████████████████████████████| 75 kB 1.6 MB/s eta 0:00:011


In [5]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [6]:
import re

def parse_bayes_output(soup_html):
    stem_re = "\d' *stem"
    rna_re = "[AUGC]+"

    seqs = [el.text for el in html_soup.select('tr')[2:]]
    seqs = [(re.findall(rna_re, mirna), re.findall(stem_re, mirna)) for mirna in seqs]
    seqs = [(strand[0], sequence[0][:2]) for strand, sequence in seqs if len(strand) > 0 and len(sequence) > 0]
    
#     output = []
#     for strand_match, sequence_match in seqs:
#         strand = strand_match[0]
#         sequence = sequence_match[0]
#         print(sequence)
#         if len(strand) > 0 and len(sequence) > 0:
#             output.append((strand, sequence))
#         seqs = [(strand[0], sequence[0][:2]) for strand, sequence in seqs if len(strand) > 0 and len(sequence) > 0]
    return list(set(seqs))

In [7]:
mature_bayes_url = "http://mirna.imbb.forth.gr/cgi-bin/MatureBayes/maturebayes.py"
mature_mirnas = []

for i in tqdm(range(filtered_premirnas.shape[0])):
    premirna_seq = filtered_premirnas.iloc[i]['sequence']
    print(premirna_seq)
    payload = {"single_seq": premirna_seq}
    
    response = requests.post(mature_bayes_url, payload)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    new_mature_mirnas = parse_bayes_output(html_soup)
    mature_mirnas += new_mature_mirnas
    
mature_mirnas = list(set(mature_mirnas))
print(f'Found {len(mature_mirnas)} mature miRNA candidates.')

  0%|          | 0/3 [00:00<?, ?it/s]

AUGCGGAAUUAUAUAGGACAGAAUAAUCAGCAACACAGUUGCUGAUUCUCUUCCUGUUCCAAGCAU
AUUGUGUGAAUUUGGACAUGUUCUUCAGGCUCAUCAACAAUUUUAUUGUAGAUGAAGAAGGUAACAUGUUCAACACCAGU
GGUUUUAUAGCUGGCUUGAUUGCCAUAGUAAUGGUGACAAUUAUGCUUUGCUGUAUGACC
Found 9 mature miRNA candidates.


In [8]:
mature_mirnas = pd.DataFrame(mature_mirnas, columns=['sequence', 'strand'])
mature_mirnas

,sequence,strand
0,UGUAGAUGAAGAAGGUAACAUG,3'
1,AUUGCCAUAGUAAUGGUGACAA,5'
2,AUGAAGAAGGUAACAUGUUCAA,3'
3,UCAACAAUUUUAUUGUAGAUGA,5'
4,CGGAAUUAUAUAGGACAGAAUA,5'
5,UUUAUAGCUGGCUUGAUUGCCA,5'
6,UGAUUCUCUUCCUGUUCCAAGC,3'
7,ACAAUUAUGCUUUGCUGUAUGA,3'
8,UCAGCAACACAGUUGCUGAUUC,5'


In [9]:
mature_mirnas.to_csv('mature-mirnas-from-vmir.csv', index=False)